# Angular (Only) Particle Filter

This particle filter is designed to use angular measures only.  Reason for this is because that is what a single camera 
can detect centroids of colored blobs.  Of course without parallax or similar, this doesn't have a chance of detecting 
anything other than the angles.  This makes some aspects of the problem such as computing weight updates simpler, but 
it makes the design of the landmarks more critical and introduces more opportunity for both higher error and the 
potential for ambiguous localization.  This is a toy project that is just demonstrating the use of a very simple case of
such a particle filter to get a sense of its fitness for certain applications.  Also, it is an experiment with a different 
style of implementation.

In [57]:
import math
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
%matplotlib inline

# Generate a few landmarks and define robot location and view
landmarks = [(0.0, 1.0), (1.0, 0.0)]
robot_loc = (0.0, 0.0)
robot_dir = 0.0

# Determine the angles for the landmarks from robot's perspective
def find_angle(landmark, test_loc, test_dir):
    delta_x = landmark[0] - test_loc[0]
    delta_y = landmark[1] - test_loc[1]
    angle = math.atan2(delta_y, delta_x)
    return angle - test_dir

angles = []
for l in landmarks:
    angles.append(find_angle(l, robot_loc, robot_dir))

#print(angles)
    
# Generate a bunch of candidate particles
particles = []
for i in range(200):
    x = np.random.uniform(-1.0, 1.0)
    y = np.random.uniform(-1.0, 1.0)
    a = np.random.uniform(-math.pi, math.pi)
    particles.append((x, y, a))
    
#print(particles)

    
for k in range(1000):    
    
    # Determine score for each of the particles
    score = []
    mvn = multivariate_normal(mean=0.0, cov=0.5)
    for p in particles:
        # Compute angles
        x, y, a = p
        p_angles = []
        for l in landmarks:
            p_angles.append(find_angle(l, (x,y), a))
        
        # Correlation
        diffs = []
        for p_a in p_angles:
            smallest_dist = 1.0e99
            for a in angles:
                dist = p_a - a
                if dist < smallest_dist:
                    smallest_dist = dist
            diffs.append(smallest_dist)
        
        # Compute score
        w = 1.0
        for d in diffs:
            w *= mvn.pdf(d)
        score.append(w)

    # Normalize score
    total_score = 0.0
    for s in score:
        total_score += s
    score = [s / total_score for s in score]

    #print(score)

    # Resample the particles
    new_particles = []
    indices = [i for i, _ in enumerate(score)]
    for i in indices:
        selected = np.random.choice(indices, p=score)
        new_particles.append(particles[selected])

    #print(new_particles)

    # Add a little noise to the particles
    particles = []
    for p in new_particles:
        x, y, a = p
        x += np.random.normal(0.0, 0.01)
        y += np.random.normal(0.0, 0.01)
        a += np.random.normal(0.0, 0.01)
        particles.append((x, y, a))
    
#print(particles) 

# Determine average location and angle

x_total = 0.0
y_total = 0.0
a_total = 0.0
total = len(particles)
for p in particles:
    x_total += p[0]
    y_total += p[1]
    a_total += p[2]
    
print("x", x_total / total)
print("y", y_total / total)
print("a", a_total / total)

('x', 1.5922222795328747)
('y', -0.780091387572847)
('a', 0.6933197240320794)
